This file explores the integration between listenbrainz, musicbrainz, and spotify.

It's separate from the application logic because the approach requires exploration to optimize and understand.
There's a lot more edge cases integratiing multiple edge cases than I had to contend with in using only Spotify.

Below, I sort out those edge cases, explore likelihood of error cases, so I can encode these states in tests.

Import Libs

In [1]:
import os
import time
import json
import requests
from requests import HTTPError
from requests.auth import HTTPBasicAuth

Set up Script Variables

In [2]:
artists = ["Noah Gundersen", "Julien Baker", "Phoebe Bridgers"]

Load Environment Variables

In [3]:
listenbrainz_api_key = os.getenv("LISTENBRAINZ_API_KEY")
spotify_client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
spotify_client_id = os.getenv("SPOTIFY_CLIENT_ID")
# listenbrainz_api_key, spotify_client_secret, spotify_client_id

Provide useful context around request errors

In [4]:
def handle_error(response):
    try:
        response.raise_for_status()
    except HTTPError as error:
        error.add_note(f"Request: {error.request}")
        error.add_note(f"Request headers: {error.request.headers}")
        error.add_note(f"Request body: {error.request.body}")
        error.add_note(f"Response: {error.response}")
        error.add_note(f"Response headers: {error.response.headers}")
        error.add_note(f"Response body: {error.response.text}")
        raise

Get Spotify Auth Token

In [5]:
spotify_auth = requests.post(
    url="https://accounts.spotify.com/api/token",
    auth=HTTPBasicAuth(spotify_client_id, spotify_client_secret),
    data={"grant_type": "client_credentials"}
)
handle_error(spotify_auth)

auth_json = spotify_auth.json()
spotify_token = auth_json['access_token']
# spotify_token

Get playlist based on artists

In [6]:
lb_radio = requests.get(
    url="https://api.listenbrainz.org/1/explore/lb-radio",
    params={
        "mode": "easy",
        "prompt": " ".join([f'artist:({artist})' for artist in artists])
    },
    headers={"Authorization": f"Bearer {listenbrainz_api_key}"}
)
handle_error(lb_radio)

tracks = lb_radio.json()["payload"]["jspf"]["playlist"]["track"]
len(tracks), tracks[:2]

(50,
 [{'album': 'Songs of Anarchy: Volume 4',
   'creator': 'Katey Sagal & The Forest Rangers',
   'duration': 328000,
   'extension': {'https://musicbrainz.org/doc/jspf#track': {'artist_identifiers': ['971e2203-5f5c-4ac1-a3e2-210413391d09',
      'a77e45f9-7f01-4577-9c1b-f2045d15aa74'],
     'release_identifier': 'https://musicbrainz.org/release/8a837ab6-c1b1-4ae9-a614-51fa4da5e3a1'}},
   'identifier': ['https://musicbrainz.org/recording/8cd815da-144c-42d0-b56c-56296f09abf0'],
   'title': 'Greensleeves'},
  {'album': 'the rest',
   'creator': 'boygenius',
   'duration': 254981,
   'extension': {'https://musicbrainz.org/doc/jspf#track': {'artist_identifiers': ['3ceeddbd-fba5-4bdb-99f7-2d028ed5afda'],
     'release_identifier': 'https://musicbrainz.org/release/a1e4415e-9691-400b-9199-46cb1048737a'}},
   'identifier': ['https://musicbrainz.org/recording/0eaec872-253d-4e0d-ab0d-6efb21b0c29a'],
   'title': 'Powers'}])

In [7]:
identifiers = []
for track in tracks:
    if len(track['identifier']) > 1:
        raise Exception(f'{track["title"]} by {track["creator"]} has multiple identifiers: {", ".join(track["identifiers"])}')
    identifiers += track["identifier"]
mbids = [x.split("/")[-1] for x in identifiers]
identifiers[:2], mbids[:2]

(['https://musicbrainz.org/recording/8cd815da-144c-42d0-b56c-56296f09abf0',
  'https://musicbrainz.org/recording/0eaec872-253d-4e0d-ab0d-6efb21b0c29a'],
 ['8cd815da-144c-42d0-b56c-56296f09abf0',
  '0eaec872-253d-4e0d-ab0d-6efb21b0c29a'])

In [8]:
all_relations = []
all_isrcs = []
spotify_tracks = []
rate_limit_remaining = 900  # Arbitrary > 0 number
rate_limit_max = 1000 # Slightly larger arbitrary > 0 number
for mbid in mbids:
    time.sleep((rate_limit_max - rate_limit_remaining) / 1000)
    
    # https://musicbrainz.org/doc/MusicBrainz_API#Lookups
    mb_recording = requests.get(
        url=f"https://musicbrainz.org/ws/2/recording/{mbid}",
        params={"fmt": "json", "inc": "isrcs url-rels artists"},
        headers={"User-Agent": "mixtapestudy.com/0.0 ( douglas@builtonbits.com )"}
    )
    handle_error(mb_recording)
    
    recording_json = mb_recording.json()
    rate_limit_remaining = int(mb_recording.headers["X-RateLimit-Remaining"])
    rate_limit_max = int(mb_recording.headers["X-RateLimit-Limit"])
    relations = recording_json["relations"]
    isrcs = recording_json["isrcs"]
    artist_credit = recording_json["artist-credit"]

    if isrcs:
        query_string = f"isrc:{isrcs[0]}"
    elif relations:
        raise Exception(f"Finally found a relation: {relations}, this never happened before")
    else:
        query_string = f'track:{recording_json["title"]}'
        if artist_credit:
            query_string += " " + " ".join([f'artist:{artist["artist"]["name"]}' for artist in artist_credit])
        
    print(f"query_string: {query_string}")
    
    # https://developer.spotify.com/documentation/web-api/reference/search
    spotify_search = requests.get(
        url="https://api.spotify.com/v1/search",
        params={"type": "track", "q": query_string},
        headers={"Authorization": f"Bearer {spotify_token}"}
    )
    handle_error(spotify_search)

    spotify_json = spotify_search.json()
    # print([t["id"] for t in spotify_json["tracks"]["items"]])
    if spotify_json["tracks"] and spotify_json["tracks"]["items"]:
        spotify_tracks.append(spotify_json["tracks"]["items"][0])
    
    all_relations += relations
    all_isrcs += isrcs
    
    # break

(len(all_relations), len(all_isrcs), len(spotify_tracks)), all_relations[:2], all_isrcs[:2]

query_string: isrc:USQX91402218
query_string: isrc:USUG12306653
query_string: isrc:US64G1210033
query_string: isrc:GBK3W2302748
query_string: track:memories artist:Soccer Mommy
query_string: isrc:USCJY1831956
query_string: track:Tomorrow (live) artist:Daughter
query_string: track:The Song That Changed Everything artist:Mac Miller artist:SZA
query_string: isrc:GBCEJ2100217
query_string: isrc:USMTD2200424
query_string: track:Day 5 artist:Japanese Breakfast
query_string: track:Carolina Rain (live in Porto) artist:Ryan Adams
query_string: track:"A really thick Scottish accent" artist:Julien Baker
query_string: isrc:USJ5G2130701
query_string: isrc:USHM81309671
query_string: track:Pray to God (Calvin Harris vs Mike Pickering Hacienda remix) artist:Calvin Harris artist:HAIM
query_string: track:Wading in Waist-high Water (Robin pitched stem) artist:Fleet Foxes
query_string: track:song 2 my journal artist:Clairo
query_string: track:Send Someone Away (radio) artist:Embee artist:José González
que

((0, 19, 28), [], ['USQX91402218', 'USUG12306653'])

In [9]:
[
    (st["id"], st["uri"], st["name"], 
    [a["name"] for a in st["artists"]])
    for st in spotify_tracks
]

[('6FGbv2o2tZslzkYXWi2268',
  'spotify:track:6FGbv2o2tZslzkYXWi2268',
  'Greensleeves - From Sons of Anarchy',
  ['Katey Sagal', 'The Forest Rangers']),
 ('4GgkYUOZareQyCK6ohJj7v',
  'spotify:track:4GgkYUOZareQyCK6ohJj7v',
  'Powers',
  ['boygenius']),
 ('6h06Uuqs2ANAhijKhZUMUq',
  'spotify:track:6h06Uuqs2ANAhijKhZUMUq',
  'Do You Hear What I Hear?',
  ['Sufjan Stevens']),
 ('5divLEEbAisvkOPrp4Vnpb',
  'spotify:track:5divLEEbAisvkOPrp4Vnpb',
  'Break In The Weather',
  ['Benjamin Francis Leftwich']),
 ('5ckUacHVEsmgJI7akKHadD',
  'spotify:track:5ckUacHVEsmgJI7akKHadD',
  'Out Of The Woods - Commentary',
  ['Taylor Swift']),
 ('2RDGPOGcOY8vXRjqbmbYM5',
  'spotify:track:2RDGPOGcOY8vXRjqbmbYM5',
  'Tomorrow (Live)',
  ['Daughter']),
 ('5ChhBJdogYlf0mrGmJGsk5',
  'spotify:track:5ChhBJdogYlf0mrGmJGsk5',
  'Magic Trick',
  ['Noah Gundersen']),
 ('5mtW8Fo5sjFQY6nA10mDIy',
  'spotify:track:5mtW8Fo5sjFQY6nA10mDIy',
  'Believe - Spotify Singles',
  ['Lucy Dacus']),
 ('0IziocrutMdtLHorPTaDyh',
  